In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET

In [30]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [32]:
# Data directory
train_data = "train_dataset"
test_data = "test_dataset"

# Batch_size
fixed_batch_size = 32

# Load dataset
train_dataset = datasets.ImageFolder(root=train_data, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_data, transform=train_transform)

# Data loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=fixed_batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=fixed_batch_size, shuffle=False, num_workers=4)

# Class to index mapping
print(train_dataset.class_to_idx)

{'area': 0, 'heatmap': 1, 'horizontal_bar': 2, 'horizontal_interval': 3, 'line': 4, 'manhattan': 5, 'map': 6, 'pie': 7, 'scatter': 8, 'scatter-line': 9, 'surface': 10, 'venn': 11, 'vertical_bar': 12, 'vertical_box': 13, 'vertical_interval': 14}


In [4]:
# Load the pre-trained model
model = models.resnet152(pretrained=True)

# Freezing all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:
        param.requires_grad = True # unfreeze final classification layer
    else:
        param.requires_grad = False

# Modify final classification layer
num_classes = 15
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

# Move model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/home/fuubian/.virtualenvs/py3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fuubian/.virtualenvs/py3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")

Epoch 1, Loss: 0.7685
Epoch 2, Loss: 0.6503
Epoch 3, Loss: 0.6159
Epoch 4, Loss: 0.6074
Epoch 5, Loss: 0.5864
Epoch 6, Loss: 0.5887
Epoch 7, Loss: 0.5699
Epoch 8, Loss: 0.5812
Epoch 9, Loss: 0.5934
Epoch 10, Loss: 0.5582
Training complete!


In [34]:
# Model evaluation
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 80.11%


In [6]:
# Saving the model
torch.save(model.state_dict(), "chart_classification_model.pth")